In [1]:
import threading

In [17]:
def fn():
    print(
        f"{threading.active_count()=},\n"
        f"{threading.current_thread()=},\n"
        f"{threading.current_thread().name=},\n"
        f"{threading.current_thread().is_alive()=},\n"
        f"{threading.get_ident()=},\n"
        f"{threading.get_native_id()=},"
    )

In [18]:
th = threading.Thread(target=fn, name="class_thread_01")
th.start()
th.join()
print(th.daemon, th.is_alive())

threading.active_count()=9,
threading.current_thread()=<Thread(class_thread_01, started 123145494265856)>,
threading.current_thread().name='class_thread_01',
threading.current_thread().is_alive()=True,
threading.get_ident()=123145494265856,
threading.get_native_id()=66687623,
False False


In [13]:
threading.enumerate()

[<_MainThread(MainThread, started 4601767424)>,
 <Thread(IOPub, started daemon 123145375129600)>,
 <Heartbeat(Heartbeat, started daemon 123145391919104)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145409781760)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 123145426571264)>,
 <ControlThread(Control, started daemon 123145443360768)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145460150272)>,
 <ParentPollerUnix(Thread-2, started daemon 123145477476352)>]

In [27]:
%%time

N = 10 ** 8


def counter(a, b):
    while a < b:
        a += 1
        
counter(0, N)

CPU times: user 4.53 s, sys: 46.7 ms, total: 4.58 s
Wall time: 4.95 s


In [22]:
%%time

N = 10 ** 8


def counter(a, b):
    while a < b:
        a += 1
        

th = threading.Thread(target=counter, name="class_thread_01", args=(0, N))
th.start()
th.join()


CPU times: user 4.7 s, sys: 43.2 ms, total: 4.75 s
Wall time: 5.12 s


In [28]:
%%time

N_THREADS = 100
COUNT_SIZE = N // N_THREADS


threads = [
    threading.Thread(
        target=counter,
        name=f"class_thread_{i}",
        args=(i * COUNT_SIZE, COUNT_SIZE * (i + 1))
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 4.55 s, sys: 103 ms, total: 4.65 s
Wall time: 4.84 s


In [29]:
from urllib.request import urlopen

In [38]:
%%time

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
URLS = [URL] * N


def fetch_url(url):
    res = urlopen(url)
    return res


def fetch_several_urls(urls):
    for url in urls:
        fetch_url(url)
        
        
fetch_several_urls(URLS)

CPU times: user 1.01 s, sys: 108 ms, total: 1.12 s
Wall time: 16.8 s


In [39]:
%%time


th = threading.Thread(target=fetch_several_urls, name="class_thread_01", args=(URLS,))
th.start()
th.join()


CPU times: user 1.02 s, sys: 115 ms, total: 1.14 s
Wall time: 17.2 s


In [44]:
%%time

N_THREADS = 25
COUNT_SIZE = N // N_THREADS


threads = [
    threading.Thread(
        target=fetch_several_urls,
        name=f"class_thread_{i}",
        args=([URL] * COUNT_SIZE,)
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 952 ms, sys: 123 ms, total: 1.07 s
Wall time: 1.16 s


In [52]:
import time

N_THREADS = 100

global_count = [0]


def update_count(n):
    for _ in range(n):
        c = global_count[0]
        time.sleep(0.000000001)
        c += 1
        global_count[0] = c
        

threads = [
    threading.Thread(
        target=update_count,
        name=f"class_thread_{i}",
        args=(100,)
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()


print(global_count, global_count[0] == 100 * 100)

[302] False


In [55]:
import time

N_THREADS = 100

global_count = [0]


def update_count(n, lock):
    for _ in range(n):
        lock.acquire()
        
        c = global_count[0]
        time.sleep(0.000000001)
        c += 1
        global_count[0] = c
    
        lock.release()

        
lock = threading.Lock()        

threads = [
    threading.Thread(
        target=update_count,
        name=f"class_thread_{i}",
        args=(100, lock)
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()


print(global_count, global_count[0] == 100 * 100)

[10000] True


In [59]:
%%time

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
URLS = [URL] * N

N_THREADS = 25
COUNT_SIZE = N // N_THREADS

SIM_CONNECTIONS = 2


def fetch_url(url, sem):
    with sem:
        res = urlopen(url)

    return res


def fetch_several_urls(urls, sem):
    for url in urls:
        fetch_url(url, sem)


sem = threading.Semaphore(SIM_CONNECTIONS)

threads = [
    threading.Thread(
        target=fetch_several_urls,
        name=f"class_thread_{i}",
        args=([URL] * COUNT_SIZE, sem)
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

CPU times: user 971 ms, sys: 105 ms, total: 1.08 s
Wall time: 8.69 s


In [1]:
import queue
import threading
from urllib.request import urlopen

In [6]:
%%time

URL = "https://ru.wikipedia.org/wiki/Python"
N = 100
URLS = [URL] * N

N_THREADS = 10
COUNT_SIZE = N // N_THREADS

SIM_CONNECTIONS = 2

q = queue.Queue()
for url in URLS:
    q.put(url)


print("queue is ready")


def fetch_url(url, sem):
    th = threading.current_thread()

    with sem:
        res = urlopen(url)

    return res


def fetch_several_urls(q, sem):
    while True:
        try:
            url = q.get_nowait()
        except Exception:
            return
        q.task_done()

        fetch_url(url, sem)


sem = threading.Semaphore(SIM_CONNECTIONS)

threads = [
    threading.Thread(
        target=fetch_several_urls,
        name=f"class_thread_{i}",
        args=(q, sem)
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()
    
q.join()

queue is ready
CPU times: user 996 ms, sys: 112 ms, total: 1.11 s
Wall time: 8.54 s
